In [42]:
import librosa
import IPython, numpy as np
from ly import *
import os

In [43]:
def xml_to_ly(file, p = False):
    '''
    creates ly file to local dir
    
    INPUTS: 
    file: string to file name of XML in Testing_Data directory
    p: bool whether to print command line response
    
    
    '''
    #go to lilypond directory
#     p = os.getcwd()
#     print(p)
#     os.chdir('Lilypond_Files')
    
    #create lilypond file
    file = './Testing_Data/'+file
    cmd = 'musicxml2ly -a '+file 
    returned_value = os.system(cmd)
    if returned_value != 0:
        raise ValueError('File Not Found')
    if p ==  True:
        print('returned value:', returned_value)
    
    
#     os.chdir('../')
    
    

In [10]:
def ly_to_text(filepath):
    '''
    INPUTS:    
        filepath: string to file path of .ly file
    -----------------------------------------
    RETURNS:
        data: string of encoded text from .ly file
    '''
    with open(ly_file,encoding='cp1252') as f: #Might have to change encoding depending on how the mac version is
        data = f.read()
    return data

All nodes (instances of Item) have a ‘position’ attribute that indicates where the item starts in the source text. Almost
all items have the token that starts the expression in the ‘token’ attribute and possibly other tokens in the ‘tokens’
attribute, as a tuple.
The ‘end_position()’ method returns the position where the node (including its child nodes) ends.

In [40]:
def parse_PPOVO(bpm, PPOVO):
    '''
    INPUTS:
        bpm: beats per minute integer
        PPOVO: string of ly file containing note information
    --------------------------------
    RETURNS:
        onset_times: array of all float ground truth onset times
    '''
    time = 0.0
    onset_times = []
    prev = None
    i = 0
    is_tuple = False    
    tuple_fraction = 1
    #parse through PPOVO section
    while i < len(PPOVO)-2:
        #c -> current char in .ly text
        c = PPOVO[i]
        c_next = PPOVO[i+1]
        c_next_dig = c_next.isdigit()
        
        if(c == "\\"): #check for override
            i+=1
            tuplecheck = PPOVO.find("override TupletBracket",i) #check for tuple override
            if(tuplecheck == i): #tuple override found
                i = PPOVO.find("times",i)
                i += 6 #skip to find ft
                n_buffer = ""
                d_buffer = ""
                c = PPOVO[i]
                
                while c != "/": #get ft numerator
                    if(c.isdigit()):
                        n_buffer += c
                    i +=1
                    c = PPOVO[i]                  
                
                while c != " ": #get ft denominator
                    if(c.isdigit()):
                        d_buffer += c
                    i +=1
                    c = PPOVO[i]
                tuple_fraction = float(n_buffer)/float(d_buffer)
                is_tuple = True
                
        elif(is_tuple and c == "}"): #check for end of tuple overide, reset values
            is_tuple = False
            tuple_fraction = 1
            
        elif(c == "'" or (c == "r" and c_next_dig)): #note or rest found    
            #add note onset
            t = None
            if(c == "'"):
                t = "note"
                onset_times.append(time)               
                #parse through note length indicator
                while(c == "'"):
                    i += 1
                    c = PPOVO[i]
            else:
                t = "rest"
                i+=1
                c = PPOVO[i]
                
            #buffer to get note/rest value
            buffer = ""
            #keep going until total note len is found (1,2,4,8th,16th,32th note etc)
            while (c.isdigit()):
                buffer+=c
                i+=1
                c = PPOVO[i]
                
            if(c == "."): #dotted note
                tuple_fraction = 1.5
                
            #convert buffer to int to get note type
            if(buffer != ""):
                note_val = int(buffer)
                duration = note_to_seconds(bpm, note_val,tf=tuple_fraction)
                time+=duration
                
            if(c == "."): #dotted note
                tuple_fraction = 1.0
#             print("buffer:",buffer)
#             print("Type:",t,"Length:",note_val,"Duration:",duration)
        i+=1
            
    return np.array(onset_times)
            
                
        

        
    

In [12]:
def note_to_seconds(bpm, note_val,tf=1):
    '''
    INPUTS: 
        beats per minute integer
        note_val: type of note in float 
            1.0 = whole note
            0.5 = half note
            etc...
        tf: tuple fraction indicated in ly file
        
    -------------------------------
    RETURNS:
        duration: note duration in seconds
    '''
    duration = (60.0/bpm)*(4.0/note_val)*tf
    return duration

In [13]:
def ly_onsets(bpm, data):
    '''
    INPUTS:
        bpm: beats per minute integer
        data: string of ly file
    --------------------------------
    RETURNS:
        onset_times: array of floats for time stamps ground truth onsets
    '''
    #skip header information
    start = data.find('PartPOneVoiceOne') 
    #improper file type
    if start == -1:
        raise ValueError('Improper File Format. File is not Monophonic')
        
    #adjust start to begin at PPOVO node
    start +=  16
    while data[start] != '{':
        start += 1
        
    PPOVO = data[start:(len(data)-1)]
    
    #parse through to find note section
    onset_times = parse_PPOVO(bpm, PPOVO)
    
    return onset_times
            
    

In [49]:
filename = "test8"
xml_file = filename+ ".xml"
ly_file = filename+".ly"


xml_to_ly(xml_file, p = True)
data = ly_to_text(ly_file)

true_onsets = ly_onsets(120, data)
print(true_onsets)

returned value: 0
[0.    0.5   1.    1.5   1.625 1.75  2.    2.25  2.5   3.    3.5   3.625
 3.75  3.875 4.    5.    5.25  5.5   5.625]


In [50]:
def extract_user_rhythm(signal):
    '''
    Inputs: audio signal (1D np.array)
    Outputs: times (sec) of onsets (1D np.array)

    Uses librosa's onset_detection to extract a 1D np.array
    of the onsets in an audio signal

    '''
    rhythm_arr = librosa.onset.onset_detect(signal, sr=sr, units='time')
    
    return rhythm_arr


In [51]:
mySignal, sr = librosa.load("Testing_Data/test8.wav", sr=None)
user_onsets = extract_user_rhythm(mySignal)
print(user_onsets)
print(len(user_onsets))
print(true_onsets)
print(len(true_onsets))

[0.03482993 0.510839   1.01006803 1.52090703 1.64861678 1.77632653
 2.02013605 2.26394558 2.51936508 3.0185941  3.51782313 3.64553288
 3.77324263 3.8893424  4.01705215 5.0155102  5.27092971 5.51473923
 5.64244898]
19
[0.    0.5   1.    1.5   1.625 1.75  2.    2.25  2.5   3.    3.5   3.625
 3.75  3.875 4.    5.    5.25  5.5   5.625]
19
